# Working with Azure Blob Storage and Machine Learning
## Wine Quality Prediction Pipeline

### Project Overview

This notebook demonstrates how to integrate **Azure Blob Storage** into a typical data science workflow. We'll build a complete pipeline from data storage to machine learning predictions.

**What we'll learn:**
- Connect to Azure Blob Storage from Python
- Create containers and upload datasets
- Download and process data from the cloud
- Build a regression model (predicting continuous values)
- Store results and models back to Azure
- Best practices for cloud data management

### Why Azure Blob Storage?

**Benefits for Data Scientists:**
- 📦 Store large datasets (GB/TB) without cluttering Git repositories
- 🔄 Share data easily across team members
- 💰 Cost-effective storage (pay only for what you use)
- 🌍 Access data from anywhere (local, Codespace, Azure ML)
- 📊 Separate data from code (better organization)
- 🔐 Secure with access controls and encryption

**Use Cases:**
- Raw datasets too large for Git (> 100 MB)
- Processed data and feature engineering results
- Trained models and model artifacts
- Experiment results and predictions
- Shared datasets for team collaboration

### The Dataset: Wine Quality

We'll use the **Wine Quality Dataset** from the UCI Machine Learning Repository.

**Dataset Details:**
- Source: Portuguese "Vinho Verde" wines
- Samples: ~6,500 wines (red and white variants)
- Features: 11 physicochemical properties
  - Fixed acidity, volatile acidity, citric acid
  - Residual sugar, chlorides
  - Free and total sulfur dioxide
  - Density, pH
  - Sulphates, alcohol content
- Target: Quality score (0-10, based on sensory data)

**ML Task:** **Regression** (predicting a continuous quality score)
- This is different from Iris (classification with categories)
- We'll predict numeric values instead of classes

### Technical Setup

**Requirements:**
- Azure Storage Account (created: `rbrandsdataplayground`)
- Connection String (stored as Codespace Secret AZURE_STORAGE_CONNECTION_STRING)
- Python packages: `azure-storage-blob`, `pandas`, `scikit-learn`, `matplotlib`

**Notebook Structure:**
1. Setup & Connection to Azure Blob Storage
2. Create Container & Upload Dataset
3. Download & Explore Data
4. Data Analysis & Visualization
5. Machine Learning - Regression Model
6. Save Results & Model to Blob Storage
7. Best Practices & Cleanup

Let's get started! 🚀

## 1. Setup and Establishing Connection 

In this first step, we set up the connection to Azure Blob Storage.
We use the `azure-storage-blob` library that we've already installed.

In [ ]:
# %%
# Imports
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
from dotenv import load_dotenv

# Load environment variables from .env file (if it exists)
# If you're using Codespace Secrets, this step is optional
load_dotenv()

# %%
# Load connection string from environment variable
connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

# Verify that connection string is available
if not connection_string:
    raise ValueError(
        "Azure connection string not found! Please set:\n"
        "AZURE_STORAGE_CONNECTION_STRING\n"
        "as a Codespace Secret or in a .env file"
    )

# Create BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Extract storage account name from connection string for display
storage_account_name = connection_string.split('AccountName=')[1].split(';')[0]
print(f"✓ Connection established to Storage Account '{storage_account_name}'")

### What's happening here?

1. **Imports**: We import the necessary classes from the Azure SDK
2. **Connection String**: We load the complete connection string from the environment variable
   - This works with Codespace Secrets (recommended)
   - Also works with a `.env` file for local development
3. **Validation**: We check if the connection string is available before proceeding
4. **BlobServiceClient**: This is our main object for all operations with Azure Blob Storage

**Why use the connection string directly?**
- Simpler: Only one environment variable instead of two
- Standard practice: Azure documentation often uses connection strings
- Flexible: The connection string contains all necessary information

**Security Note**: The connection string contains sensitive data - never commit it to your repository!

In [ ]:
# %%
# Test the connection by listing all containers
print("\nExisting containers:")
try:
    containers = blob_service_client.list_containers()
    container_list = [container.name for container in containers]

    if container_list:
        for container_name in container_list:
            print(f"  - {container_name}")
    else:
        print("  (No containers yet)")

    print(f"\n✓ Connection successfully tested!")

except Exception as e:
    print(f"✗ Connection error: {str(e)}")

## 2. Creating and Managing Containers

Containers in Azure Blob Storage are similar to folders - they organize your blobs (files).
Each storage account can contain unlimited containers, and each container can hold unlimited blobs.

**Container naming rules:**
- Must be lowercase
- Can contain letters, numbers, and hyphens
- Must start with a letter or number
- Length: 3-63 characters

In [ ]:
# %%
# Define container name
container_name = "playground-data"

# Create container (if it doesn't exist)
try:
    container_client = blob_service_client.create_container(container_name)
    print(f"✓ Container '{container_name}' created successfully")
except Exception as e:
    if "ContainerAlreadyExists" in str(e):
        print(f"ℹ Container '{container_name}' already exists")
        container_client = blob_service_client.get_container_client(container_name)
    else:
        print(f"✗ Error creating container: {str(e)}")
        raise

### Checking Container Properties

Once a container is created, we can inspect its properties and metadata.

In [ ]:
# %%
# Get container properties
properties = container_client.get_container_properties()

print(f"\nContainer Properties for '{container_name}':")
print(f"  - Last Modified: {properties.last_modified}")
print(f"  - ETag: {properties.etag}")
print(f"  - Lease Status: {properties.lease.status}")
print(f"  - Lease State: {properties.lease.state}")

# %%
# List all containers in the storage account
print("\nAll containers in storage account:")
containers = blob_service_client.list_containers(include_metadata=True)
for container in containers:
    print(f"  - {container.name}")
    if container.metadata:
        print(f"    Metadata: {container.metadata}")

### Adding Metadata to Containers

Metadata is useful for storing additional information about containers, like descriptions, owners, or categories.
Metadata consists of name-value pairs and is stored with the container.

In [ ]:
# %%
# Set metadata for the container
metadata = {
    'purpose': 'learning-project',
    'environment': 'development',
    'owner': 'rbrands'
}

container_client.set_container_metadata(metadata)
print(f"✓ Metadata added to container '{container_name}'")

# Retrieve and display metadata
properties = container_client.get_container_properties()
print(f"\nContainer Metadata:")
for key, value in properties.metadata.items():
    print(f"  - {key}: {value}")

## 3. Uploading Blobs (Files)

Now let's upload some files to our container. Azure Blob Storage supports three types of blobs:
- **Block Blobs**: Optimized for uploading large amounts of data (text, images, videos, etc.)
- **Append Blobs**: Optimized for append operations (log files)
- **Page Blobs**: Optimized for random read/write operations (VHD files)

We'll focus on **Block Blobs** as they're the most commonly used type.

In [ ]:
# %%
# First, let's create some sample data to upload
import pandas as pd
import numpy as np

# Create a simple CSV file
data = {
    'id': range(1, 11),
    'name': [f'Item_{i}' for i in range(1, 11)],
    'value': np.random.randint(10, 100, 10),
    'category': np.random.choice(['A', 'B', 'C'], 10)
}

df = pd.DataFrame(data)
print("Sample data created:")
print(df.head())

# Save to local file (temporary)
local_file_path = 'sample_data.csv'
df.to_csv(local_file_path, index=False)
print(f"\n✓ Sample file created: {local_file_path}")

### Uploading a File

There are several ways to upload data to Azure Blob Storage:
1. **From a local file** (most common)
2. **From data in memory** (text, bytes)


Let's try all methods:

In [ ]:
# %%
# Method 1: Upload from local file
blob_name = "data/sample_data.csv"  # Note: we can use "folders" in the blob name

# Get a blob client
blob_client = container_client.get_blob_client(blob_name)

# Upload the file
with open(local_file_path, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

print(f"✓ File uploaded to: {blob_name}")

In [ ]:
# %%
# Method 2: Upload data directly from memory (without saving to disk first)
text_data = """
This is a sample text file created in memory.
It demonstrates uploading data without creating a local file first.
This is useful for generated content or processed data.
"""

blob_name_text = "data/sample_text.txt"
blob_client_text = container_client.get_blob_client(blob_name_text)
blob_client_text.upload_blob(text_data, overwrite=True)

print(f"✓ Text uploaded to: {blob_name_text}")

In [ ]:
# %%
# Method 3: Upload JSON data from memory
import json

json_data = {
    'experiment': 'azure-blob-test',
    'date': '2024-01-14',
    'results': [
        {'trial': 1, 'score': 0.85},
        {'trial': 2, 'score': 0.91},
        {'trial': 3, 'score': 0.88}
    ]
}

blob_name_json = "data/experiment_results.json"
blob_client_json = container_client.get_blob_client(blob_name_json)
blob_client_json.upload_blob(json.dumps(json_data, indent=2), overwrite=True)

print(f"✓ JSON uploaded to: {blob_name_json}")

### Listing Blobs in a Container

After uploading, let's verify that our files are in the container:

In [ ]:
# %%
# List all blobs in the container
print(f"\nBlobs in container '{container_name}':")
print("-" * 60)

blob_list = container_client.list_blobs()
for blob in blob_list:
    print(f"  📄 {blob.name}")
    print(f"     Size: {blob.size:,} bytes")
    print(f"     Created: {blob.creation_time}")
    print(f"     Last Modified: {blob.last_modified}")
    print()

# %%
# List blobs with a specific prefix (like a "folder")
print("Blobs in 'data/' folder:")
blob_list = container_client.list_blobs(name_starts_with="data/")
for blob in blob_list:
    print(f"  - {blob.name}")

### Blob Properties and Metadata

Just like containers, individual blobs can have properties and metadata:

In [ ]:
# %%
# Get properties of a specific blob
blob_client = container_client.get_blob_client("data/sample_data.csv")
properties = blob_client.get_blob_properties()

print(f"Properties for 'data/sample_data.csv':")
print(f"  - Content Type: {properties.content_settings.content_type}")
print(f"  - Size: {properties.size:,} bytes")
print(f"  - ETag: {properties.etag}")
print(f"  - Last Modified: {properties.last_modified}")
print(f"  - Creation Time: {properties.creation_time}")

# %%
# Set metadata for a blob
metadata = {
    'source': 'python-script',
    'data_type': 'sample',
    'version': '1.0'
}

blob_client.set_blob_metadata(metadata)
print("\n✓ Metadata added to blob")

# Retrieve metadata
properties = blob_client.get_blob_properties()
print("\nBlob Metadata:")
for key, value in properties.metadata.items():
    print(f"  - {key}: {value}")

## 4. Downloading Blobs (Files)

Now that we have files in Azure Blob Storage, let's learn how to download them.
Just like uploading, there are multiple ways to download data:
1. **Download to a local file**
2. **Download to memory** (for direct processing)
3. **Stream download** (for large files)

In [ ]:
# %%
# Method 1: Download blob to a local file
download_file_path = "downloaded_sample_data.csv"
blob_name = "data/sample_data.csv"

blob_client = container_client.get_blob_client(blob_name)

with open(download_file_path, "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())

print(f"✓ Blob downloaded to: {download_file_path}")

# Verify the download by reading the CSV
df_downloaded = pd.read_csv(download_file_path)
print(f"\nDownloaded data preview:")
print(df_downloaded.head())

### Downloading Directly to Memory

For data processing workflows, it's often more efficient to load data directly into memory
without creating intermediate files:

In [ ]:
# %%
# Method 2: Download directly to memory and process
blob_name = "data/sample_data.csv"
blob_client = container_client.get_blob_client(blob_name)

# Download as bytes
blob_data = blob_client.download_blob().readall()

# Convert bytes to pandas DataFrame directly
from io import BytesIO
df_from_blob = pd.read_csv(BytesIO(blob_data))

print("Data loaded directly from blob:")
print(df_from_blob.head())
print(f"\nShape: {df_from_blob.shape}")

In [ ]:
# %%
# Download text file to string
blob_name = "data/sample_text.txt"
blob_client = container_client.get_blob_client(blob_name)

# Download as text
text_content = blob_client.download_blob().readall().decode('utf-8')

print("Text content from blob:")
print(text_content)

In [ ]:
# %%
# Download and parse JSON
blob_name = "data/experiment_results.json"
blob_client = container_client.get_blob_client(blob_name)

# Download and parse JSON
json_content = blob_client.download_blob().readall().decode('utf-8')
experiment_data = json.loads(json_content)

print("JSON data from blob:")
print(json.dumps(experiment_data, indent=2))

# Access the data
print(f"\nExperiment: {experiment_data['experiment']}")
print(f"Number of trials: {len(experiment_data['results'])}")

### Streaming Large Files

For large files, it's more memory-efficient to stream the download in chunks
rather than loading everything into memory at once:

In [ ]:
# %%
# Method 3: Stream download (useful for large files)
blob_name = "data/sample_data.csv"
blob_client = container_client.get_blob_client(blob_name)

download_stream = blob_client.download_blob()

# Process in chunks
chunk_size = 1024  # 1KB chunks
chunks_processed = 0

print("Streaming download:")
for chunk in download_stream.chunks():
    chunks_processed += 1
    # In a real scenario, you would process each chunk here
    # For example: write to file, process data, etc.

print(f"✓ Downloaded in {chunks_processed} chunks")
print(f"  Total size: {download_stream.size:,} bytes")

### Downloading Multiple Files

Often you need to download multiple files at once. Here's an efficient way to do that:

In [ ]:
# %%
# Download all files from a "folder" (prefix)
import os

# Create a directory for downloads
download_dir = "downloads"
os.makedirs(download_dir, exist_ok=True)

# List all blobs with prefix "data/"
blob_list = container_client.list_blobs(name_starts_with="data/")

print(f"Downloading all files from 'data/' folder:")
print("-" * 60)

for blob in blob_list:
    # Create local file path
    local_path = os.path.join(download_dir, blob.name.replace("data/", ""))

    # Download blob
    blob_client = container_client.get_blob_client(blob.name)
    with open(local_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

    print(f"✓ Downloaded: {blob.name} → {local_path}")

print(f"\n✓ All files downloaded to '{download_dir}/' directory")

### Conditional Downloads

You can download files only if they meet certain conditions (e.g., modified since last download):

In [ ]:
# %%
# Download only if blob was modified after a certain date
from datetime import datetime, timedelta

blob_name = "data/sample_data.csv"
blob_client = container_client.get_blob_client(blob_name)

# Get blob properties
properties = blob_client.get_blob_properties()
last_modified = properties.last_modified

# Check if modified in the last hour (for example)
one_hour_ago = datetime.now(last_modified.tzinfo) - timedelta(hours=1)

if last_modified > one_hour_ago:
    print(f"✓ Blob was recently modified ({last_modified})")
    print("  Downloading latest version...")
    data = blob_client.download_blob().readall()
    print(f"  Downloaded {len(data):,} bytes")
else:
    print(f"ℹ Blob not modified recently (last modified: {last_modified})")
    print("  Skipping download")

## 5. Machine Learning Workflow with Azure Blob Storage

Now let's see how Azure Blob Storage fits into a real machine learning workflow.
We'll use the Wine Quality dataset and demonstrate:
1. Loading data from Azure Blob Storage
2. Training a model
3. Saving the trained model to Azure Blob Storage
4. Loading the model for predictions

This simulates a typical ML workflow where:
- Data scientists share datasets via cloud storage
- Trained models are versioned and stored centrally
- Models can be deployed from cloud storage

In [ ]:
# %%
# Import ML libraries
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import pickle

# Load the wine dataset
wine = load_wine()
X = wine.data
y = wine.target

# Create a DataFrame for better understanding
wine_df = pd.DataFrame(X, columns=wine.feature_names)
wine_df['target'] = y
wine_df['target_name'] = wine_df['target'].map({0: wine.target_names[0],
                                                   1: wine.target_names[1],
                                                   2: wine.target_names[2]})

print("Wine Dataset Overview:")
print(f"  - Samples: {len(wine_df)}")
print(f"  - Features: {len(wine.feature_names)}")
print(f"  - Classes: {len(wine.target_names)}")
print(f"\nClass distribution:")
print(wine_df['target_name'].value_counts())
print("\nFirst few rows:")
print(wine_df.head())

### Step 1: Upload Dataset to Azure Blob Storage

First, let's save and upload our dataset to Azure Blob Storage.
This simulates a scenario where data scientists share datasets via cloud storage.

In [ ]:
# %%
# Save dataset to CSV
dataset_filename = 'wine_dataset.csv'
wine_df.to_csv(dataset_filename, index=False)

# Upload to Azure Blob Storage in a 'datasets' folder
blob_name = f"ml-project/datasets/{dataset_filename}"
blob_client = container_client.get_blob_client(blob_name)

with open(dataset_filename, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

print(f"✓ Dataset uploaded to: {blob_name}")

# Add metadata
metadata = {
    'dataset': 'wine-quality',
    'samples': str(len(wine_df)),
    'features': str(len(wine.feature_names)),
    'classes': str(len(wine.target_names)),
    'uploaded_by': 'robert'
}
blob_client.set_blob_metadata(metadata)
print("✓ Metadata added to dataset")

### Step 2: Load Dataset from Azure and Train Model

Now let's load the dataset directly from Azure Blob Storage and train a model.
This demonstrates how team members can access shared datasets without local copies.

In [ ]:
# %%
# Download dataset directly from Azure Blob Storage to memory
blob_client = container_client.get_blob_client(blob_name)
blob_data = blob_client.download_blob().readall()

# Load into pandas DataFrame
wine_df_from_azure = pd.read_csv(BytesIO(blob_data))

print("✓ Dataset loaded from Azure Blob Storage")
print(f"  Shape: {wine_df_from_azure.shape}")

# Prepare features and target
X = wine_df_from_azure.drop(['target', 'target_name'], axis=1)
y = wine_df_from_azure['target']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# %%
# Train a Random Forest model
print("\nTraining Random Forest model...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"✓ Model trained successfully")
print(f"  Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=wine.target_names))

# %%
# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 5 Most Important Features:")
print(feature_importance.head())

### Step 3: Save Trained Model to Azure Blob Storage

After training, we save the model to Azure Blob Storage.
This allows:
- Model versioning
- Team collaboration
- Model deployment from a central location

In [ ]:
# %%
# Serialize the model using pickle
model_filename = 'wine_classifier_model.pkl'

# Save model locally first
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)

print(f"✓ Model serialized to: {model_filename}")

# Upload to Azure Blob Storage
model_blob_name = f"ml-project/models/{model_filename}"
blob_client = container_client.get_blob_client(model_blob_name)

with open(model_filename, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

print(f"✓ Model uploaded to: {model_blob_name}")

# Add metadata about the model
model_metadata = {
    'model_type': 'RandomForestClassifier',
    'accuracy': str(round(accuracy, 4)),
    'n_estimators': '100',
    'train_samples': str(len(X_train)),
    'test_samples': str(len(X_test)),
    'features': str(len(X.columns)),
    'trained_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

blob_client.set_blob_metadata(model_metadata)
print("✓ Model metadata added")

### Step 4: Load Model from Azure for Predictions

Now let's simulate a deployment scenario: loading the model from Azure Blob Storage
to make predictions. This is how you would use models in production.

In [ ]:
# %%
# Download and load the model from Azure Blob Storage
print("Loading model from Azure Blob Storage...")
blob_client = container_client.get_blob_client(model_blob_name)

# Get model metadata first
properties = blob_client.get_blob_properties()
print("\nModel Metadata:")
for key, value in properties.metadata.items():
    print(f"  - {key}: {value}")

# Download model
model_data = blob_client.download_blob().readall()

# Load model from bytes
loaded_model = pickle.loads(model_data)
print("\n✓ Model loaded successfully from Azure")

# %%
# Make predictions with the loaded model
print("\nTesting loaded model with new predictions:")

# Take a few samples from test set
sample_indices = [0, 5, 10]
X_sample = X_test.iloc[sample_indices]
y_sample = y_test.iloc[sample_indices]

# Predict
predictions = loaded_model.predict(X_sample)
probabilities = loaded_model.predict_proba(X_sample)

# Display results
for i, (idx, pred, prob, true) in enumerate(zip(sample_indices, predictions, probabilities, y_sample)):
    print(f"\nSample {i+1}:")
    print(f"  True class: {wine.target_names[true]}")
    print(f"  Predicted class: {wine.target_names[pred]}")
    print(f"  Confidence: {prob[pred]:.2%}")
    print(f"  All probabilities: {dict(zip(wine.target_names, [f'{p:.2%}' for p in prob]))}")

### Step 5: Model Versioning Example

In real projects, you'll train multiple versions of your model.
Let's see how to implement simple versioning:

In [ ]:
# %%
# Create a versioned model name
from datetime import datetime

version = datetime.now().strftime('%Y%m%d_%H%M%S')
versioned_model_name = f"ml-project/models/wine_classifier_v{version}.pkl"

# Upload with version in name
blob_client = container_client.get_blob_client(versioned_model_name)
with open(model_filename, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

# Add comprehensive metadata
versioned_metadata = {
    'model_type': 'RandomForestClassifier',
    'version': version,
    'accuracy': str(round(accuracy, 4)),
    'dataset': 'wine-quality',
    'n_estimators': '100',
    'random_state': '42',
    'description': 'Production model with optimized hyperparameters'
}

blob_client.set_blob_metadata(versioned_metadata)
print(f"✓ Versioned model saved: {versioned_model_name}")

# %%
# List all model versions
print("\nAll model versions in storage:")
print("-" * 60)

model_blobs = container_client.list_blobs(name_starts_with="ml-project/models/")
for blob in model_blobs:
    print(f"\n📦 {blob.name}")
    print(f"   Size: {blob.size:,} bytes")
    print(f"   Created: {blob.creation_time}")

    # Get metadata if available
    blob_client = container_client.get_blob_client(blob.name)
    properties = blob_client.get_blob_properties()
    if properties.metadata:
        print("   Metadata:")
        for key, value in properties.metadata.items():
            print(f"     - {key}: {value}")

### Summary: Complete ML Workflow

We've demonstrated a complete machine learning workflow using Azure Blob Storage:

1. ✓ **Data Storage**: Uploaded dataset to cloud storage with metadata
2. ✓ **Data Loading**: Loaded data directly from Azure into memory for training
3. ✓ **Model Training**: Trained a Random Forest classifier
4. ✓ **Model Storage**: Saved trained model to Azure with metadata
5. ✓ **Model Deployment**: Loaded model from Azure for predictions
6. ✓ **Versioning**: Implemented simple model versioning strategy

**Benefits of this approach:**
- **Collaboration**: Team members can access shared datasets and models
- **Reproducibility**: Models and data are versioned and centrally stored
- **Scalability**: No local storage limitations
- **Deployment**: Models can be loaded from anywhere with internet access
- **Tracking**: Metadata helps track model performance and lineage